
# Dashboard Interactivo — Importaciones y Consumos de Combustibles (Avance)

**Herramientas:** Voilá + Plotly + ipywidgets  
**Paleta:** Verde `#2E6E36`, Amarillo `#FFB81C`, Naranja `#E35205`, Azul `#003366` (fondo claro `#E6EEF3`)  
Este cuaderno contiene **4 visualizaciones interactivas** con **filtros enlazados** listas para correr con Voilá.


In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML

FONDO = "#E6EEF3"
PALETA = {
    "Gasolina regular": "#2E6E36",
    "Gasolina superior": "#FFB81C",
    "Diesel alto azufre": "#E35205",
    "AzulOscuro": "#003366"
}

def apply_bg(fig):
    fig.update_layout(plot_bgcolor=FONDO, paper_bgcolor=FONDO)
    return fig


In [ ]:

# === Carga de datos desde tu Excel ===
importacion = pd.read_excel("Datos.xlsx", sheet_name= "IMPORTACION", header=6) 
importacion = importacion.iloc[:288].reset_index(drop=True)

consumo = pd.read_excel("Datos.xlsx", sheet_name= "CONSUMO", header=6) 
consumo = consumo.iloc[:300].reset_index(drop= True)

importaciones = importacion[["Fecha", "Gasolina regular", "Gasolina superior", "Diesel alto azufre"]].copy()
consumos = consumo[["Fecha", "Gasolina regular", "Gasolina superior", "Diesel alto azufre"]].copy()

for df in (importaciones, consumos):
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df["Año"] = df["Fecha"].dt.year
    df["Mes_num"] = df["Fecha"].dt.month
    df["Mes"] = df["Fecha"].dt.month_name()
    df.sort_values("Fecha", inplace=True)

YEARS_ALL = sorted(list(set(importaciones["Año"].unique()).union(consumos["Año"].unique())))
MESES_ORD = list(range(1,13))
NOMBRES_MESES = ['January','February','March','April','May','June','July','August','September','October','November','December']

display(HTML(f"<b>Importaciones:</b> {importaciones.shape[0]} filas — <b>Consumos:</b> {consumos.shape[0]} filas"))


In [ ]:

# === Filtros ===
dataset_toggle = widgets.ToggleButtons(
    options=[("Importaciones","importaciones"), ("Consumos","consumos")],
    value="importaciones",
    description="Datos:"
)

year_dropdown = widgets.Dropdown(
    options=YEARS_ALL,
    value=min(YEARS_ALL),
    description="Año:"
)

combustibles_sel = widgets.SelectMultiple(
    options=["Gasolina regular","Gasolina superior","Diesel alto azufre"],
    value=("Gasolina regular","Gasolina superior","Diesel alto azufre"),
    description="Combustibles:",
    rows=3
)

mes_dropdown = widgets.Dropdown(
    options=[(NOMBRES_MESES[m-1], m) for m in MESES_ORD],
    value=1,
    description="Mes:"
)

year_range = widgets.IntRangeSlider(
    value=(min(YEARS_ALL), max(YEARS_ALL)),
    min=min(YEARS_ALL), max=max(YEARS_ALL), step=1,
    description='Rango años:',
    continuous_update=False
)

display(widgets.HBox([dataset_toggle, year_dropdown, combustibles_sel, mes_dropdown]))
display(year_range)


In [ ]:

def current_df():
    return importaciones if dataset_toggle.value == "importaciones" else consumos


In [ ]:

def viz1_barras_mensuales(año, combustibles):
    df = current_df()
    tmp = (df[df["Año"]==año]
           .groupby("Mes_num")[list(combustibles)].mean()
           .reindex(range(1,13))
           .reset_index())
    tmp["Mes"] = [NOMBRES_MESES[m-1] for m in tmp["Mes_num"]]

    fig = go.Figure()
    for c in combustibles:
        fig.add_bar(x=tmp["Mes"], y=tmp[c], name=c, marker_color=PALETA[c])
    fig.update_layout(
        title=f"Promedio mensual por tipo — {año} ({'Importaciones' if dataset_toggle.value=='importaciones' else 'Consumos'})",
        barmode="group", xaxis_title="Mes", yaxis_title="Volumen"
    )
    return apply_bg(fig)


In [ ]:

def viz2_pie_share(año):
    df = current_df()
    cols = ["Gasolina regular","Gasolina superior","Diesel alto azufre"]
    vals = df[df["Año"]==año][cols].mean()
    fig = px.pie(values=vals.values, names=vals.index, color=vals.index,
                 color_discrete_map=PALETA,
                 title=f"Distribución porcentual — {año}")
    return apply_bg(fig)


In [ ]:

def viz3_serie_tiempo(year_min, year_max, combustibles):
    df = current_df()
    tmp = df[(df["Año"]>=year_min) & (df["Año"]<=year_max)].copy()
    fig = go.Figure()
    for c in combustibles:
        fig.add_scatter(x=tmp["Fecha"], y=tmp[c], mode="lines", name=c, line=dict(width=2.5, color=PALETA[c]))
    fig.update_layout(title=f"Tendencia temporal {year_min}-{year_max}", xaxis_title="Fecha", yaxis_title="Volumen")
    return apply_bg(fig)


In [ ]:

import plotly.io as pio
def viz4_heatmap(col):
    df = current_df()
    pivot = (df.groupby(["Año","Mes_num"])[col].mean()
               .unstack(fill_value=0).reindex(index=YEARS_ALL, columns=MESES_ORD))
    fig = px.imshow(pivot, aspect="auto", labels=dict(x="Mes", y="Año", color="Promedio"),
                    x=[NOMBRES_MESES[m-1] for m in MESES_ORD], y=YEARS_ALL,
                    title=f"Heatmap mensual por año — {col}")
    return apply_bg(fig)

combustible_heatmap = widgets.Dropdown(
    options=["Gasolina regular","Gasolina superior","Diesel alto azufre"],
    value="Gasolina regular",
    description="Heatmap:"
)
display(combustible_heatmap)


In [ ]:

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

def render_all(*args):
    out1.clear_output(wait=True)
    out2.clear_output(wait=True)
    out3.clear_output(wait=True)
    out4.clear_output(wait=True)
    with out1:
        display(viz1_barras_mensuales(year_dropdown.value, combustibles_sel.value))
    with out2:
        display(viz2_pie_share(year_dropdown.value))
    with out3:
        yr0, yr1 = year_range.value
        display(viz3_serie_tiempo(yr0, yr1, combustibles_sel.value))
    with out4:
        display(viz4_heatmap(combustible_heatmap.value))

for w in (dataset_toggle, year_dropdown, combustibles_sel, mes_dropdown, year_range, combustible_heatmap):
    w.observe(render_all, names="value")

display(
    widgets.VBox([
        widgets.HTML(f"<h3 style='color:{PALETA['AzulOscuro']}'>Dashboard — Avance</h3>"),
        widgets.HBox([out1, out2]),
        widgets.HBox([out3]),
        widgets.HBox([out4])
    ])
)

render_all()
